In [34]:
import wrangle
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.linear_model import LinearRegression,LassoLars,TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error,explained_variance_score

In [14]:
df = wrangle.bee_merged()
df.head()

,state,year,season,beekeepers,total_loss,average_loss,starting_colonies,colonies_lost,ending_colonies,beekeepers_exclusive_to_state,colonies_exclusive_to_state,ansi,latitude,longitude
0,alabama,2022,annual,33,36.488812,34.260096,316,212,369,100.000000,100.000000,1,32.806671,-86.791130
1,arkansas,2022,annual,18,51.254480,53.867865,152,143,136,94.444444,97.368421,5,34.969704,-92.373123
2,arkansas,2022,annual,17,49.411765,52.869897,148,126,129,100.000000,100.000000,5,34.969704,-92.373123
3,california,2022,annual,89,33.269667,42.818791,166009,85526,171543,67.415730,25.320314,6,36.116203,-119.681564
4,california,2022,annual,29,36.752854,35.811393,123975,73971,127295,0.000000,0.000000,6,36.116203,-119.681564


In [15]:

def split_data(df):
    '''This function will input dataframe and split into train,validate,test'''
    #split dataframw into 80% train  and 20% test 
    train_validate, test = train_test_split(df, test_size=.2, random_state=825)
     #split train further into 75% train, 25% validate
    train, validate = train_test_split(train_validate, test_size=.25, random_state=825)
    
    #return train,validate,test back to function
    return train,validate,test

In [20]:
#split data
train,validate,test = split_data(df)
train.shape,validate.shape,test.shape

((628, 14), (210, 14), (210, 14))

In [32]:
def scale_data(train,validate,test,columns):
    #make the scaler
    scaler = MinMaxScaler()
    #fit the scaler at train data only
    scaler.fit(train[columns])
    #tranforrm train, validate and test
    train_scaled = scaler.transform(train[columns])
    validate_scaled = scaler.transform(validate[columns])
    test_scaled = scaler.transform(test[columns])
    
    # Generate a list of the new column names with _scaled added on
    scaled_columns = [col+"_scaled" for col in columns]
    
    #concatenate with orginal train, validate and test
    scaled_train = pd.concat([train.reset_index(drop = True),pd.DataFrame(train_scaled,columns = scaled_columns)],axis = 1)
    scaled_validate = pd.concat([validate.reset_index(drop = True),pd.DataFrame(validate_scaled, columns = scaled_columns)], axis = 1)
    scaled_test= pd.concat([test.reset_index(drop = True),pd.DataFrame(test_scaled,columns = scaled_columns)],axis = 1)
    
    return scaled_train,scaled_validate,scaled_test

In [35]:
columns = [col for col in df.drop(columns = ["state","season"])]
scaled_train,scaled_validate,scaled_test = scale_data(train,validate,test,columns)

In [38]:
scaled_train.head(1)

,state,year,season,beekeepers,total_loss,average_loss,starting_colonies,colonies_lost,ending_colonies,beekeepers_exclusive_to_state,...,total_loss_scaled,average_loss_scaled,starting_colonies_scaled,colonies_lost_scaled,ending_colonies_scaled,beekeepers_exclusive_to_state_scaled,colonies_exclusive_to_state_scaled,ansi_scaled,latitude_scaled,longitude_scaled
0,michigan,2015,annual,206,27.117985,53.774448,7497,2788,7493,100.0,...,0.189463,0.573333,0.021262,0.010846,0.024308,1.0,1.0,0.454545,0.787362,0.712385


In [39]:
def select_rfe(X,y,  n_features_to_select = 5):
    #create the model
    rfe=RFE(LinearRegression(), n_features_to_select = n_features_to_select) 
    #fit the model
    rfe.fit(X,y)
    #use get_support()
    return X.columns[rfe.get_support()]

In [ ]:
X = df

In [6]:
def get_baseline_RMSE(y_train):
    '''
    this function will calculate baseline mean and baseline median and calculate RMSE from mean and median
    '''
    #get mean of target from train
    y_train["baseline_mean"] = y_train.colonies_lost.mean()
    #get median of target from train
    y_train["baseline_median"] =y_train.colonies_lost.median()
    #get mean of target from validate
    y_validate["baseline_mean"] = y_validate.colonies_lost.mean()
    #get median from target from validate
    y_validate["baseline_median"] =y_validate.colonies_lost.median()
    
    #calculate RMSE 
    RMSE_train_mean=mean_squared_error(y_train.colonies_lost,y_train.baseline_mean, squared = False)
    RMSE_validate_mean=mean_squared_error(y_validate.colonies_lost,y_validate.baseline_mean, squared = False)

    print("RMSE using Mean on \nTrain: ", round(RMSE_train_mean,2), "\nValidate: ", round(RMSE_validate_mean,2))
    print()

    #calculate RMSE
    RMSE_train_median= mean_squared_error(y_train.colonies_lost,y_train.baseline_median, squared = False)
    RMSE_validate_median= mean_squared_error(y_validate.colonies_lost,y_validate.baseline_median, squared = False)

    print("RMSE using Median on \nTrain: ", round(RMSE_train_median,2), "\nValidate: ", round(RMSE_validate_median,2))

In [7]:
get_baseline_RMSE(y_train)

RMSE using Mean on 
Train:  25321.86 
Validate:  21956.59

RMSE using Median on 
Train:  26950.17 
Validate:  23132.97


In [10]:
#make a dataframe to capture model and RMSE 
metric_df = pd.DataFrame(data=[
            {
                'model': 'Baseline', 
                'RMSE_train': rmse_train_mean,
                'RMSE_validate': rmse_validate_mean
                }
            ])

NameError: name 'rmse_train_mean' is not defined

In [9]:
# create the model object
lm = LinearRegression(normalize = True)
# Fit the model
lm.fit(X_train, y_train.colonies_lost)
# Predict y on train
y_train['colonies_lost_pred_lm'] = lm.predict(X_train)
# predict validate
y_validate['colonies_lost_pred_lm'] = lm.predict(X_validate)
    
# evaluate: train rmse
rmse_train = round(mean_squared_error(y_train.colonies_lost, y_train.colonies_lost_pred_lm,squared = False), 2)
# evaluate: validate rmse
rmse_validate = round(mean_squared_error(y_validate.colonies_lost, y_validate.colonies_lost_pred_lm,squared = False),2)

#append model and RMSE from OLS model to metric dataframe
metric_df = metric_df.append({
    'model': 'OLS Regressor', 
    'RMSE_train': rmse_train,
    'RMSE_validate': rmse_validate,
    }, ignore_index=True)

NameError: name 'metric_df' is not defined